# House Prices: 探索的データ分析 (EDA)

このノートブックでは、Kaggle の **House Prices - Advanced Regression Techniques** コンペティションのデータに対して探索的データ分析（EDA）を行います。

EDA の目的は以下のとおりです：
- データの構造と特徴を理解する
- 目的変数（SalePrice）の分布を確認する
- 欠損値のパターンを把握する
- 特徴量と目的変数の関係を可視化する
- 外れ値を特定する

---

## 0. ライブラリのインポート

まず、分析に必要なライブラリを読み込みます。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# 日本語フォントの設定（環境に応じて変更してください）
# plt.rcParams['font.family'] = 'IPAexGothic'

# グラフのスタイル設定
sns.set_theme(style='whitegrid', palette='muted')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

# pandas の表示設定
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

import warnings
warnings.filterwarnings('ignore')

print('Setup complete.')

---
## 1. データの読み込みと概要

訓練データ（train.csv）とテストデータ（test.csv）を読み込み、基本的な情報を確認します。

**確認ポイント：**
- データのサイズ（行数・列数）
- 各列のデータ型
- 先頭数行のデータ
- 基本統計量

In [ ]:
# データの読み込み
train = pd.read_csv('../data/raw/train.csv')
test = pd.read_csv('../data/raw/test.csv')

print(f'訓練データのサイズ: {train.shape[0]} 行 x {train.shape[1]} 列')
print(f'テストデータのサイズ: {test.shape[0]} 行 x {test.shape[1]} 列')
print(f'\n訓練データにのみ存在する列: {set(train.columns) - set(test.columns)}')

テストデータには `SalePrice`（目的変数）が含まれていないことが確認できます。これはコンペティションで予測すべき値です。

In [ ]:
# 先頭5行を表示
print('--- 訓練データの先頭5行 ---')
train.head()

In [ ]:
# データ型の確認
print('--- データ型の一覧 ---')
print(f'\n数値型の列数: {train.select_dtypes(include=[np.number]).shape[1]}')
print(f'カテゴリ型（object）の列数: {train.select_dtypes(include=["object"]).shape[1]}')
print()
train.dtypes

In [ ]:
# 基本統計量（数値変数）
print('--- 基本統計量 ---')
train.describe()

In [ ]:
# 基本統計量（カテゴリ変数）
print('--- カテゴリ変数の基本統計量 ---')
train.describe(include=['object'])

---
## 2. 目的変数（SalePrice）の分布

機械学習モデルを構築する前に、目的変数の分布を確認することは非常に重要です。

**なぜ分布を確認するのか？**
- 多くの回帰モデルは、目的変数が正規分布に近いほど性能が向上します
- 歪み（skewness）が大きい場合、対数変換などで正規分布に近づけることが有効です

In [ ]:
# SalePrice の基本統計量
print('--- SalePrice の基本統計量 ---')
print(train['SalePrice'].describe())
print(f'\n歪度 (Skewness): {train["SalePrice"].skew():.4f}')
print(f'尖度 (Kurtosis): {train["SalePrice"].kurt():.4f}')

歪度（Skewness）が正の値の場合、分布は右に裾が長い（右に歪んでいる）ことを示します。住宅価格は一般的に右に歪んだ分布を持ちます。高額な物件が少数存在するためです。

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# ヒストグラム（元のスケール）
axes[0].hist(train['SalePrice'], bins=50, color='steelblue', edgecolor='white', alpha=0.8)
axes[0].set_title('SalePrice Distribution', fontsize=14)
axes[0].set_xlabel('SalePrice')
axes[0].set_ylabel('Frequency')
axes[0].axvline(train['SalePrice'].mean(), color='red', linestyle='--', label=f'Mean: ${train["SalePrice"].mean():,.0f}')
axes[0].axvline(train['SalePrice'].median(), color='orange', linestyle='--', label=f'Median: ${train["SalePrice"].median():,.0f}')
axes[0].legend()

# 対数変換後のヒストグラム
log_saleprice = np.log1p(train['SalePrice'])
axes[1].hist(log_saleprice, bins=50, color='seagreen', edgecolor='white', alpha=0.8)
axes[1].set_title('log(1 + SalePrice) Distribution', fontsize=14)
axes[1].set_xlabel('log(1 + SalePrice)')
axes[1].set_ylabel('Frequency')

# Q-Qプロット（対数変換後）
stats.probplot(log_saleprice, plot=axes[2])
axes[2].set_title('Q-Q Plot (log transformed)', fontsize=14)

plt.tight_layout()
plt.show()

print(f'対数変換後の歪度: {log_saleprice.skew():.4f}')
print(f'対数変換後の尖度: {log_saleprice.kurt():.4f}')

**観察結果：**
- 元の SalePrice は右に歪んだ分布を持っています
- 対数変換（`log1p`）を適用すると、正規分布にかなり近づきます
- Q-Qプロットでも、対数変換後のデータが理論上の正規分布の直線に沿っていることが確認できます

> **Tips:** Kaggle の多くの上位解法では、`SalePrice` に対数変換を適用してからモデルを学習させています。

---

## 3. 欠損値の確認

欠損値の扱いは特徴量エンジニアリングの重要なステップです。欠損値のパターンを理解することで、適切な補完方法を選択できます。

**注意：** このデータセットでは、一部の `NA` は「該当なし」（例：ガレージがない場合の GarageType）を意味しており、本当の欠損ではないケースがあります。

In [ ]:
# 訓練データの欠損値を集計
missing_train = train.isnull().sum()
missing_train = missing_train[missing_train > 0].sort_values(ascending=False)
missing_train_pct = (missing_train / len(train) * 100).round(2)

missing_df = pd.DataFrame({
    'Missing Count': missing_train,
    'Missing %': missing_train_pct
})

print(f'欠損値を持つ列の数: {len(missing_df)}')
print(f'全列数: {train.shape[1]}')
print()
missing_df

In [ ]:
# 欠損値の可視化（欠損率が高い順にバーチャート）
fig, ax = plt.subplots(figsize=(12, 6))

colors = ['#e74c3c' if pct > 50 else '#f39c12' if pct > 10 else '#3498db'
          for pct in missing_train_pct.values]

bars = ax.bar(range(len(missing_train_pct)), missing_train_pct.values, color=colors, edgecolor='white')
ax.set_xticks(range(len(missing_train_pct)))
ax.set_xticklabels(missing_train_pct.index, rotation=45, ha='right')
ax.set_ylabel('Missing Percentage (%)')
ax.set_title('Missing Values by Feature (Training Data)', fontsize=14)

# 凡例の追加
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#e74c3c', label='> 50% missing'),
    Patch(facecolor='#f39c12', label='10-50% missing'),
    Patch(facecolor='#3498db', label='< 10% missing')
]
ax.legend(handles=legend_elements, loc='upper right')

# 各バーの上に割合を表示
for bar, pct in zip(bars, missing_train_pct.values):
    if pct > 5:
        ax.text(bar.get_x() + bar.get_width()/2., bar.get_height() + 0.5,
                f'{pct:.0f}%', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

**観察結果：**
- `PoolQC`, `MiscFeature`, `Alley`, `Fence`, `FireplaceQu` は欠損率が非常に高い（これらは「該当する設備がない」ことを意味する場合が多い）
- `LotFrontage` は約18%の欠損があり、近隣の中央値などで補完するのが一般的です
- `GarageType`, `GarageYrBlt` などのガレージ関連は、ガレージがない物件で欠損になっています
- `BsmtQual`, `BsmtCond` などの地下室関連も同様です

---

## 4. 数値変数の相関分析

数値変数間の相関を調べ、`SalePrice` と強い相関を持つ特徴量を特定します。

**相関係数の解釈：**
- |r| > 0.7 : 強い相関
- 0.4 < |r| < 0.7 : 中程度の相関
- |r| < 0.4 : 弱い相関

In [ ]:
# SalePrice との相関係数を計算し、上位を表示
numeric_cols = train.select_dtypes(include=[np.number]).columns
correlation = train[numeric_cols].corr()['SalePrice'].drop('SalePrice').sort_values(ascending=False)

print('--- SalePrice との相関係数 (上位15) ---')
print(correlation.head(15).to_string())
print()
print('--- SalePrice との相関係数 (下位5) ---')
print(correlation.tail(5).to_string())

In [ ]:
# SalePrice との相関係数を棒グラフで表示
fig, ax = plt.subplots(figsize=(10, 8))

colors = ['#e74c3c' if v > 0.5 else '#3498db' if v > 0 else '#95a5a6'
          for v in correlation.values]

correlation.plot(kind='barh', color=colors, edgecolor='white', ax=ax)
ax.set_title('Correlation with SalePrice (Numeric Features)', fontsize=14)
ax.set_xlabel('Pearson Correlation Coefficient')
ax.axvline(x=0, color='black', linewidth=0.5)
ax.axvline(x=0.5, color='red', linewidth=0.5, linestyle='--', alpha=0.5)
ax.axvline(x=-0.5, color='red', linewidth=0.5, linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# 上位10個の特徴量 + SalePrice のヒートマップ
top_features = correlation.head(10).index.tolist()
top_features.append('SalePrice')

fig, ax = plt.subplots(figsize=(12, 10))
corr_matrix = train[top_features].corr()

mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdBu_r',
            center=0, vmin=-1, vmax=1, square=True, linewidths=0.5, ax=ax)
ax.set_title('Correlation Heatmap (Top 10 Features + SalePrice)', fontsize=14)

plt.tight_layout()
plt.show()

**観察結果：**
- `OverallQual`（全体的な品質）が SalePrice と最も強い相関を持っています
- `GrLivArea`（地上の居住面積）、`GarageCars`（ガレージの車台数）、`GarageArea`（ガレージ面積）も強い相関を示しています
- `GarageCars` と `GarageArea` は互いに強い相関があります（多重共線性）。モデルによっては片方だけ使う方が良い場合があります
- `TotalBsmtSF` と `1stFlrSF` も互いに強い相関があります

---

## 5. 重要な特徴量の可視化

相関の高い特徴量について、SalePrice との散布図を作成して関係性を視覚的に確認します。

In [ ]:
# SalePrice と相関の高い特徴量の散布図
key_features = ['OverallQual', 'GrLivArea', 'GarageCars', 'TotalBsmtSF',
                'GarageArea', '1stFlrSF', 'FullBath', 'TotRmsAbvGrd',
                'YearBuilt']

fig, axes = plt.subplots(3, 3, figsize=(16, 14))
axes = axes.flatten()

for i, feature in enumerate(key_features):
    ax = axes[i]
    ax.scatter(train[feature], train['SalePrice'], alpha=0.3, s=15, color='steelblue')
    ax.set_xlabel(feature)
    ax.set_ylabel('SalePrice')
    ax.set_title(f'{feature} vs SalePrice (r={train[feature].corr(train["SalePrice"]):.3f})', fontsize=11)

    # 回帰直線を追加
    z = np.polyfit(train[feature].dropna(), train.loc[train[feature].notna(), 'SalePrice'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(train[feature].min(), train[feature].max(), 100)
    ax.plot(x_line, p(x_line), color='red', linewidth=1.5, alpha=0.7)

plt.suptitle('Key Numeric Features vs SalePrice', fontsize=16, y=1.01)
plt.tight_layout()
plt.show()

**観察結果：**
- `OverallQual` は SalePrice と明確な正の関係があります。品質が上がるにつれて価格も上がる傾向が見られます
- `GrLivArea` は概ね線形的な関係ですが、右下にいくつかの外れ値が確認できます（広いのに安い物件）
- `YearBuilt` は新しい物件ほど価格が高い傾向がありますが、散らばりも大きいです
- `GarageCars` は離散的な値を取り、車2-3台分のガレージがある物件が高価です

---

## 6. カテゴリ変数の確認

主要なカテゴリ変数について、SalePrice との関係をボックスプロットで確認します。

**ボックスプロットの読み方：**
- 箱の中央の線 = 中央値
- 箱の上端・下端 = 第3四分位数・第1四分位数
- ひげ = 箱の1.5倍の範囲
- 点 = 外れ値

In [ ]:
# 主要なカテゴリ変数のボックスプロット
cat_features = ['MSZoning', 'Neighborhood', 'BldgType', 'HouseStyle',
                'ExterQual', 'KitchenQual', 'BsmtQual', 'GarageFinish']

fig, axes = plt.subplots(4, 2, figsize=(16, 22))
axes = axes.flatten()

for i, feature in enumerate(cat_features):
    ax = axes[i]

    # カテゴリごとの中央値でソート
    order = train.groupby(feature)['SalePrice'].median().sort_values().index

    sns.boxplot(data=train, x=feature, y='SalePrice', order=order,
                ax=ax, palette='viridis', fliersize=2)
    ax.set_title(f'{feature} vs SalePrice', fontsize=12)
    ax.set_xlabel(feature)
    ax.set_ylabel('SalePrice')

    # x軸ラベルが多い場合は回転
    if len(order) > 6:
        ax.tick_params(axis='x', rotation=45)

plt.suptitle('Categorical Features vs SalePrice', fontsize=16, y=1.01)
plt.tight_layout()
plt.show()

**観察結果：**
- `ExterQual`（外装品質）と `KitchenQual`（キッチン品質）はカテゴリごとに SalePrice の差が明確です。`Ex`（Excellent）が最も高く、`Fa`/`Po` が低い
- `Neighborhood`（地域）によって価格帯が大きく異なります。NoRidge, NridgHt, StoneBr が高価格帯
- `BldgType` では `1Fam`（一戸建て）が最も多く、`TwnhsE` も比較的高い価格帯です

---

In [ ]:
# Neighborhood を詳しく見る（物件数と価格の中央値）
neighborhood_stats = train.groupby('Neighborhood')['SalePrice'].agg(['median', 'count', 'mean']).sort_values('median', ascending=False)
neighborhood_stats.columns = ['Median Price', 'Count', 'Mean Price']

fig, ax1 = plt.subplots(figsize=(14, 6))

color_map = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(neighborhood_stats)))
bars = ax1.bar(range(len(neighborhood_stats)), neighborhood_stats['Median Price'],
               color=color_map, edgecolor='white')
ax1.set_xticks(range(len(neighborhood_stats)))
ax1.set_xticklabels(neighborhood_stats.index, rotation=45, ha='right')
ax1.set_ylabel('Median SalePrice ($)', color='green')
ax1.set_title('Neighborhood: Median SalePrice & Count', fontsize=14)

# 第2軸に物件数を追加
ax2 = ax1.twinx()
ax2.plot(range(len(neighborhood_stats)), neighborhood_stats['Count'],
         color='navy', marker='o', markersize=4, linewidth=1.5, alpha=0.7)
ax2.set_ylabel('Number of Houses', color='navy')

plt.tight_layout()
plt.show()

---
## 7. 外れ値の確認

外れ値はモデルの性能に大きな影響を与える可能性があります。特に `GrLivArea`（地上居住面積）に注目して外れ値を確認します。

**外れ値の検出方法：**
- 散布図による視覚的確認
- IQR（四分位範囲）法
- ドメイン知識に基づく判断

In [ ]:
# GrLivArea vs SalePrice（外れ値の特定）
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 左: 外れ値をハイライト
ax = axes[0]
ax.scatter(train['GrLivArea'], train['SalePrice'], alpha=0.4, s=20, color='steelblue', label='Normal')

# GrLivArea が 4000 以上で SalePrice が低い物件を外れ値としてマーク
outliers = train[(train['GrLivArea'] > 4000) & (train['SalePrice'] < 300000)]
ax.scatter(outliers['GrLivArea'], outliers['SalePrice'], color='red', s=80,
           marker='X', zorder=5, label=f'Outliers (n={len(outliers)})')

for _, row in outliers.iterrows():
    ax.annotate(f'Id={int(row["Id"])}',
                xy=(row['GrLivArea'], row['SalePrice']),
                xytext=(10, 10), textcoords='offset points',
                fontsize=9, color='red',
                arrowprops=dict(arrowstyle='->', color='red', lw=1))

ax.set_xlabel('GrLivArea (sq ft)')
ax.set_ylabel('SalePrice ($)')
ax.set_title('GrLivArea vs SalePrice - Outlier Detection', fontsize=13)
ax.legend()

# 右: OverallQual vs SalePrice（外れ値をハイライト）
ax = axes[1]
ax.scatter(train['OverallQual'], train['SalePrice'], alpha=0.4, s=20, color='steelblue')

# 低品質なのに高額、または高品質なのに低額な物件
outlier_high = train[(train['OverallQual'] <= 4) & (train['SalePrice'] > 200000)]
outlier_low = train[(train['OverallQual'] >= 9) & (train['SalePrice'] < 200000)]

ax.scatter(outlier_high['OverallQual'], outlier_high['SalePrice'],
           color='red', s=80, marker='X', zorder=5, label='Low quality, high price')
ax.scatter(outlier_low['OverallQual'], outlier_low['SalePrice'],
           color='orange', s=80, marker='X', zorder=5, label='High quality, low price')

ax.set_xlabel('OverallQual')
ax.set_ylabel('SalePrice ($)')
ax.set_title('OverallQual vs SalePrice - Outlier Detection', fontsize=13)
ax.legend()

plt.tight_layout()
plt.show()

print('--- GrLivArea の外れ値候補 ---')
print(outliers[['Id', 'GrLivArea', 'SalePrice', 'OverallQual', 'Neighborhood']].to_string(index=False))

In [ ]:
# 複数の特徴量でのボックスプロット（外れ値の確認）
outlier_features = ['LotArea', 'GrLivArea', 'TotalBsmtSF', '1stFlrSF',
                    'GarageArea', 'SalePrice']

fig, axes = plt.subplots(2, 3, figsize=(16, 10))
axes = axes.flatten()

for i, feature in enumerate(outlier_features):
    ax = axes[i]
    data = train[feature].dropna()
    bp = ax.boxplot(data, vert=True, patch_artist=True,
                    boxprops=dict(facecolor='lightblue', color='steelblue'),
                    medianprops=dict(color='red', linewidth=2),
                    flierprops=dict(marker='o', markerfacecolor='red', markersize=3, alpha=0.5))
    ax.set_title(f'{feature}', fontsize=12)
    ax.set_ylabel('Value')

    # 外れ値の数を表示
    Q1 = data.quantile(0.25)
    Q3 = data.quantile(0.75)
    IQR = Q3 - Q1
    n_outliers = ((data < Q1 - 1.5 * IQR) | (data > Q3 + 1.5 * IQR)).sum()
    ax.text(0.95, 0.95, f'Outliers: {n_outliers}',
            transform=ax.transAxes, ha='right', va='top',
            fontsize=10, color='red',
            bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.suptitle('Boxplots for Outlier Detection (IQR Method)', fontsize=14, y=1.01)
plt.tight_layout()
plt.show()

**観察結果：**
- `GrLivArea` が 4000 sq ft を超えるのに SalePrice が低い2つの物件は明確な外れ値です。多くの Kaggle 解法ではこれらを削除しています
- `LotArea`（敷地面積）にも極端に大きな値を持つ物件がいくつか存在します
- 外れ値の削除は慎重に行う必要があります。テストデータにも同様の物件が含まれている可能性があるためです

> **Tips:** 外れ値を削除するかどうかはモデルや検証結果に基づいて判断しましょう。一般的に、GrLivArea > 4000 & SalePrice < 300000 の物件は削除することが推奨されています。

---
## まとめ

この EDA を通じて得られた主な知見をまとめます。

### 目的変数
- `SalePrice` は右に歪んだ分布を持ち、対数変換が有効

### 欠損値
- `PoolQC`, `MiscFeature`, `Alley`, `Fence` は欠損率が非常に高い（「該当なし」の意味）
- ガレージ関連・地下室関連の欠損は、それらの設備がないことを示している
- `LotFrontage` は近隣の値で補完するのが一般的

### 重要な特徴量
- **最重要:** `OverallQual`, `GrLivArea`, `GarageCars`, `GarageArea`, `TotalBsmtSF`
- `Neighborhood`, `ExterQual`, `KitchenQual` といったカテゴリ変数も重要
- `GarageCars` と `GarageArea` には多重共線性がある

### 外れ値
- `GrLivArea` > 4000 かつ `SalePrice` < 300000 の2物件は削除候補

### 次のステップ
1. 欠損値の補完（特徴量エンジニアリング）
2. 外れ値の処理
3. 特徴量の作成（合計面積、築年数など）
4. モデルの構築と評価